<h1>3D Truss Analysis Example</h1>
<p>This document demonstrates using Jupyter for vector operations by determinging the axial forces in a simple 3D space truss.</p>
<h2>Initialization</h2>
<p>The code below imports the required modules into the notebook.  We use <a href = https://docs.scipy.org/doc/numpy-dev/user/quickstart.html>numpy</a> for its <a href = https://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.array.html#numpy.array>arrays</a>.  The <code>nump.linalg</code> gives us <a href = https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html>norm</a> to determine a vectors magnitude.  I use <code>X=0, Y=1, Z=2</code> as index values to return the comonents of a vector.</p>

In [1]:
from numpy import *
from numpy.linalg import *
set_printoptions(precision=3, suppress=True)
X=0
Y=1
Z=2

<h2>Problem Statement</h2>
<p><img src="3D Space Truss.jpg" style="float:right;width:3.5in;">For the image shown to the right, determine all axial forces as well as all reaction forces.  The applied force is $F=\left( 0, 40kN, 0 \right)$</p>

<p>We define all the ponits for use later in computing unit vectors.  We also define a function to make this a trivial task.</p>

In [2]:
a=array((1.1, -0.4, 0))
b=array((1,0,0))
c=array((0,0,0.6))
d=array((0,0,-0.4))
e=array((0,0.8,0))

Fapp=array((0, 40, 0))

def λ(a,b):
    return (b-a)/norm(b-a)

<h2>Joint A</h2>
<p>I should have an FBD of joint A here, but I will save that for another update.<p>
<p>In vector form, force equilibrium at joint A gives $T_{AB} \lambda_{AB}+T_{AC} \lambda_{AC}+T_{AD} \lambda_{AD}+F_{app}=0$.  We need to calculate the unit vectors.</p>

In [3]:
λab=λ(a,b)
λac=λ(a,c)
λad=λ(a,d)
print('The unit vectors λab, λac, λad are:', '\n',λab, '\n',λac,'\n', λad)

The unit vectors λab, λac, λad are: 
 [-0.243  0.97   0.   ] 
 [-0.836  0.304  0.456] 
 [-0.889  0.323 -0.323]


<p>We can write the above equilibrium equation in matrx form as $M \centerdot T = -F_{app}$.</p>
<p>In this case, $$T=\left[ \begin{array}{c} T_{AB} \\ T_{AC} \\ T_{AD} \end{array} \right]$$
and $M$ is composed of the unit vectors as defined in the cell below.</p>

In [4]:
m=matrix(((λab[X],λac[X],λad[X]),
          (λab[Y],λac[Y],λad[Y]),
          (λab[Z],λac[Z],λad[Z])))
print(m)

[[-0.243 -0.836 -0.889]
 [ 0.97   0.304  0.323]
 [ 0.     0.456 -0.323]]


<p>The solution is then:
$$\left[ \begin{array}{c} T_{AB} \\ T_{AC} \\ T_{AD} \end{array} \right]=M^{-1} \centerdot \left(-F_{app}\right)$$which we calculate in the following cells.</p>

In [5]:
minv=inv(m)
print(minv)

[[ 0.412  1.134 -0.   ]
 [-0.526 -0.132  1.315]
 [-0.742 -0.186 -1.237]]


In [6]:
answer=minv.dot(-Fapp)
(TAB, TAC, TAD)=answer.tolist()[0]
print("The results are TAB={:.3f}, TAC={:.3f} and TAD={:.3f}.".format(TAB, TAC, TAD))

The results are TAB=-45.354, TAC=5.261 and TAD=7.422.


<h2>Joint B</h2>
<p>We proceed in the exact same way at this joint.  The only difference is that the known force comes from TAB, not an externally applied force.</p>

In [9]:
λba=λ(b,a)
λbc=λ(b,c)
λbd=λ(b,d)
λbe=λ(b,e)
print(λba, '\n',λbc,'\n', λbd,'\n', λbe)

[ 0.243 -0.97   0.   ] 
 [-0.857  0.     0.514] 
 [-0.928  0.    -0.371] 
 [-0.781  0.625  0.   ]


In [10]:
m=matrix(((λbc[X],λbd[X],λbe[X]),
          (λbc[Y],λbd[Y],λbe[Y]),
          (λbc[Z],λbd[Z],λbe[Z])))
print(m)

[[-0.857 -0.928 -0.781]
 [ 0.     0.     0.625]
 [ 0.514 -0.371  0.   ]]


In [11]:
minv=inv(m)
print(minv)

[[-0.466 -0.583  1.166]
 [-0.646 -0.808 -1.077]
 [ 0.     1.601  0.   ]]


In [12]:
FonBfromA=TAB * λba #note this is the unit vector from B to A, not A to B.
print(FonBfromA)

[-11.  44.  -0.]


In [13]:
answer=minv.dot(-FonBfromA)
(TBC, TBD, TBE)=answer.tolist()[0]
print("The results are TBC={:.3f}, TBD={:.3f} and TBE={:.3f}.".format(TBC, TBD, TBE))

The results are TBC=20.525, TBD=28.434 and TBE=-70.434.


<h2>Joint E</h2>
<p>In this case, the equilibrium equation only has the known axial force from BE and the three reaction forces at E.  This makes the equilibrium equation very simple.</p> 

In [15]:
λeb=-λbe
m=identity(3)
print(m)

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


In [16]:
minv=inv(m)
minv

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [17]:
FonEfromBE= TBE * λeb
print(FonEfromBE)

[-55.  44.   0.]


In [19]:
E=dot(minv,-FonEfromBE)
print(E)

[ 55. -44.   0.]


<h2>Joint C</h2>

In [21]:
λca=-λac
λcb=-λbc

In [24]:
FonCfromAC=TAC * λca
FonCfromBC=TBC * λcb
print('The force on C from member AC is {} \n''The force on C from member BC is {}'
.format(FonCfromAC, FonCfromBC))

The force on C from member AC is [ 4.4 -1.6 -2.4] 
The force on C from member BC is [ 17.6   -0.   -10.56]


In [26]:
C=dot(minv,-(FonCfromAC+FonCfromBC))
print(C)

[-22.     1.6   12.96]


<h2>Joint D</h2>

In [28]:
λda=-λad
λdb=-λbd

In [29]:
D=dot(minv,-(TAD * λda + TBD * λdb))
print(D)

[-33.     2.4  -12.96]


<p>We can now check our work.  The sum of the reaction forces should equal the applided force.</p>

In [30]:
Freaction=C + D + E
print(Freaction)

[  0. -40.   0.]


<p>This is the negative of the applided force</p>

In [32]:
print(Fapp)

[ 0 40  0]
